### Mesh processing for SimuCell3D

This notebook outlines how to generate a geometry for the SimuCell3D.

In [ ]:
import os
import numpy as np
from misc import get_valid_cell_ids
from MeshPrep import convert_filtered_meshes, mesh_process_clean, string_to_array

#### 1. Select patch of cells for Simulations

To select smaller patches of cells for simulation one may load the labeled image in *napari* and select cells manually, or load meshes on *paraview* and use the existing tools (*suggested*). To see how to use paraview look [here](Tutorials/LabelSelection.md).

Before extracting a patch of cells, it is helpful to exclude all the cells that are cut or touch the border.

In [ ]:
ROOT_DIR = '/path/to/statistics/collection/output/for/a/tissue'

filtered_cell_list = get_valid_cell_ids(os.path.join(ROOT_DIR, 'cell_stats/stats_dataset_tissue_name.csv'))

In [ ]:
# The directory from which mesh files are loaded in stl format
source_mesh_path = os.path.join(ROOT_DIR, 'cell_meshes')

# The directory in which filtered meshes will be saved in vtk format
dest_mesh_path = '/path/to/dir/where/to/save/filtered/and/converted/meshes' 

convert_filtered_meshes(source_mesh_path, dest_mesh_path, filtered_cell_list)

Now you can open the `.vtk` mesh files in paraview and select a clump of cells for simulation.

#### 2. Mesh Refinemnent for SimuCell3D
SimuCell3D pipeline requires rather smooth and regular meshes to work propetly.
The following cell is meant to generate and refine meshes for SimuCell3D simulation framework. 

NOTE: Ensure that the path to labels is of cleaned, processed labels. 

In [ ]:
ROOT_DIR = '/path/to/statistics/collection/output/for/a/tissue'
voxel_resolution = np.array([0.1, 0.1, 0.1])
label_path = os.path.join(ROOT_DIR, 'processed_labels.tif')
stats_df_path = os.path.join(ROOT_DIR, 'cell_stats//stats_dataset_tissue_name.csv')

label_list = string_to_array("1 2 3 4 5")
output_dir = '/path/to/output/dir/for/clean/meshes'

# Call the mesh_process_clean function
mesh_process_clean(
    label_path=label_path, 
    output_dir=output_dir, 
    label_list=label_list, 
    voxel_resolution=voxel_resolution,
    path_to_cell_stats_df=stats_df_path, 
    scale_factor=1e-6, 
    min_edge_length=0.5,
    make_shell=True,
    shell_type="from_mesh",
    displace_shell_pts=False,
    inter_meshes=False,
)

NOTE: To create a shell mesh that fits  really well the single cell meshes you may need to resort to the simulation pipeline.

In that case you need to do the following:
- In the source code (./include/mesh/cell_types), set for ECM cells `is_static=False` in the constructors, while set it to `True` for the epithelial cells.
- Set surface tension values for ECM in at the middle of the "feasible range" experimented for epithelial cells (e.g., 1e-3 could work).
- Run few simulation iterations untile the result is satisfactory. Take the resulting vtk file as the input for following simulation runs.